<a href="https://colab.research.google.com/github/MMRES-PyBootcamp/MMRES-python-bootcamp/blob/master/12_PCA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Session 12 - Principal Component Analysis (55')
> Extra content to provide a detailed approach to the *Principal Component Analysis* (PCA). Here you will learn the fundamental the concepts on which PCA is based by means a quite exhaustive approach that mixes very gradual constructions, both analytical and intuitive.

## Outline
 * [Introduction](#Introduction)
 * [Loading a dataset](#Loading-a-dataset)
 * [Variance and covariance](#Variance-and-covariance)
 * [The covariance matrix](#The-covariance-matrix)
 * [The essence behind of the covariance matrix](#The-essence-behind-of-the-covariance-matrix)
   * [Standardizing the covariance matrix](#Standardizing-the-covariance-matrix)
   * [Eigenthings of the covariance matrix](#Eigenthings-of-the-covariance-matrix)
   * [The meaning of the covariance matrix](#The-meaning-of-the-covariance-matrix)
 * [Principal component analysis](#Principal-component-analysis)
 * [Dimensionality reduction](#Dimensionality-reduction)
 * [PCA step by step (from scratch)](#PCA-step-by-step-(from-scratch))
   * [1) Standardize data](#1-Standardize-data)
   * [2) Get covariance matrix and its eigenthings](#2-Get-covariance-matrix-and-its-eigenthings)
   * [3) Sort PCs and compute explained variance per PC](#3-Sort-PCs-and-compute-explained-variance-per-PC)
   * [4) Get the projection matrix and define number of PCs to keep](#4-Get-the-projection-matrix-and-define-number-of-PCs-to-keep)
   * [5) Transform the data](#5-Transform-the-data)
   * [6) Visualize the PCA](#6-Visualize-the-PCA)
 * [PCA (with scikit-learn)](#PCA-(with-scikit-learn))

This document is devised as a tool to enable your **self-learning process**. If you get stuck at some step or need any kind of help, please don't hesitate to raise your hand and ask for the teacher's guidance. Along it, you will find some **special cells**:

<div class="alert alert-block alert-success"><b>Practice:</b> Practice cells announce exercises that you should try during the current boot camp session. Usually, solutions are provided using hidden cells (look for the dot dot dot symbol "..." and click to unravel them and check that your try is correct).
</div>

<div class="alert alert-block alert-warning"><b>Extension:</b> Extension cells correspond to exercises (or links to contents) that are a bit more advanced. We recommend to try them after the current boot camp session.
</div>

<div class="alert alert-block alert-info"><b>Tip:</b> Tip cells just give some advice or complementary information.
</div>

<div class="alert alert-block alert-danger"><b>Caveat:</b> Caveat cells warn you about the most common pitfalls one founds when starts his/her path learning Python.

</div>

---

## Introduction

The [**Principal Component Analysis**](https://en.wikipedia.org/wiki/Principal_component_analysis) (PCA), despite being a relatively simple and classical concept, still is a very powerful tool in almost any field dealing with high-dimensionality data. This [**dimensionality reduction**](https://en.wikipedia.org/wiki/Dimensionality_reduction) strategy allows us to switch from a dataset of arbitrarily high dimensionality, to a lower dimensionality (for example, 2D) that condensates the information according their sources of variability.

For example, imagine that we have 100 patients and we measure their levels for *2 blood proteins* (let's say, albumins and globulins). The dimensionality of this data is $(100 \times 2)$ and can be completely visualized by means of a typical 2D scatter plot (100 points in a two-dimensional space). This standard visualization would be enough to start getting insights about the patients (general trends, clusters of patients with similar levels albumin/globulin, outlier patients, etc...). Imagine now that we are able to measure their levels for *1000 blood proteins*. In this case, the resulting dataset has $(10 \times 1000)$ dimensionality and now is much harder to get insights (100 points in a one-hundred-dimensional space).

The PCA allows us to embed this high-dimensionality $(10 \times 1000)$ into just a few dimensions that capture the big majority of dataset variability. This new dimensions are called *Principal Components* (PCs), and are sorted in increasing order according the total dataset variability they account for. Usually, a typical 2D scatter plot between the first two PCs (PC2 vs PC1) is more than enough to assimilate the essence of the whole dataset at a quick glance.

Let's start by loading all the packages we will need along the way:

In [ ]:
# Load packages with their corresponding alias
import numpy as np
import pandas as pd
import plotnine as p9

## Loading a dataset

Let's stats by loading again the "IQ" dataset [online resource](https://lectures.scientific-python.org/packages/statistics/index.html).

In [ ]:
# Loading the "IQ" dataset directly from an URL
df = pd.read_csv(
    filepath_or_buffer='https://scipy-lectures.org/_downloads/brain_size.csv',
    sep=';',
    na_values=".",
    index_col=0
)

# Show df's head
df.head()

Remeber that the columns *FSIQ*, *VIQ* and *PIQ* stand for different "intelligence quotients" and the *MRI_Count* is another "intelligence" measure based on magnetic resonance imaging. The columns *Gender*, *Weight* and *Height* express the gender, the weight (in pounds) and the height (in inches) of the individuals, respectively. As usual, let's express *Weight* and *Height* column values to metric units before starting.

In [ ]:
# Convert from imperial units to metric units
df['Weight [kg]'] = df['Weight'] * 0.45359237
df['Height [m]'] = df['Height'] * 0.0254

# Drop imperial columns
df.drop(columns=['Weight', 'Height'], inplace=True)

# Show df's head
df.head()

## Variance and covariance

Suppose that we have random variable $x = \left( x_i, \dots, x_n \right)$, then, its *variance* is defined as the *expected value of the squared deviation between $x$ and its expected value $E[x]$*:

\begin{equation}
\sigma^2_x = E \left[ \left( x - E \left[ x \right] \right)^2  \right],
\end{equation}

which in the particular case of a discrete equally likely random variable becomes: 

\begin{equation}
\sigma^2_x = \frac{1}{N} \sum_{i=1}^n{\left( x_i - \mu_x \right)^2}
\end{equation}.

Let's try to implement this last $\sigma^2_{x}$ expression using *NumPy*:

In [ ]:
# Initiate a "seeded" random number generator
rnd_seed = np.random.default_rng(seed=1985)

# Generate a random x variable following a uniform distribution
x = rnd_seed.uniform(low=-1, high=+1, size=15)

# Compute the variance of x by hand
sigma2_x = (1 / len(x)) * sum( (x - x.mean())**2 )

# Show the result
print(sigma2_x)

Before continuing, let's quickly check if our variance implementation is correct:

In [ ]:
# Compute the variance of x with NumPy and show it
np.var(x)

So far so good, now let's move on! The *covariance* of two random variables $x = \left( x_i, \dots, x_n \right)$ and $y = \left( y_i, \dots, y_n \right)$ is defined as the *expected value of the product of the deviations between those variables ($x$ and $y$) and their corresponding expected values ($E[x]$ and $E[y]$)*:

\begin{equation}
\sigma^2_{xy} = E \left[ \sigma^2_x \sigma^2_y \right] = E \left[ \left( x - E \left[ x \right] \right) \left( y - E \left[ y \right] \right) \right],
\end{equation}


which in the particular case of a discrete equally likely random variable becomes: 

\begin{equation}
\sigma^2_{xy} = \frac{1}{N} \sum_{i=1}^n{\left( x_i - \mu_x \right) \left( y_i - \mu_y \right)}
\end{equation}.

As we just did for $\sigma^2_{x}$, let's try to implement the $\sigma^2_{xy}$ expression using **NumPy**:

In [ ]:
# Initiate a "seeded" random number generator
rnd_seed = np.random.default_rng(seed=1985)

# Generate random x and y variables following a uniform distribution
x = rnd_seed.uniform(low=-1, high=+1, size=15)
y = rnd_seed.uniform(low=-1, high=+1, size=15)

# Compute the covariance between x and y by hand
sigma2_xy = (1 / len(x)) * sum( (x - x.mean()) * (y - y.mean()) )
print(sigma2_xy)

And, again, let's quickly check our result:

In [ ]:
# Compute the covariance between x and y with NumPy and show it
np.cov(x, y, bias=True)

## The covariance matrix

Note that when we asked **NumPy** to compute the covariance between $x$ and $y$, we got a $2 \times 2$ symmetric matrix. The off-diagonal elements in this matrix are the covariance between $x$ and $y$ ($\sigma^2_{xy}$) and the covariance between $y$ and $x$ ($\sigma^2_{yx}$), which, as expected, gave identical estimates. Analogously, the diagonal elements in this matrix are the covariance of $x$ with itself ($\sigma^2_{xx}$, AKA the variance of $x$, $\sigma^2_{x}$) and the covariance of $y$ with itself ($\sigma^2_{yy}$, AKA the variance of $y$, $\sigma^2_{y}$):

\begin{equation}
 \begin{pmatrix}
 0.3125 & -0.0888 \\
-0.0888 & 0.2506
\end{pmatrix} = \begin{pmatrix}
 \sigma^2_{xx} & \sigma^2_{xy} \\
\sigma^2_{yx} & \sigma^2_{yy}
\end{pmatrix} = \begin{pmatrix}
 \sigma^2_{x} & \sigma^2_{xy} \\
\sigma^2_{yx} & \sigma^2_{y}
\end{pmatrix}
\end{equation}

Let's check this out:

In [ ]:
# Compute the covariance between x and y by hand (sigma2_xx)
sigma2_xx = (1 / len(x)) * sum( (x - x.mean()) * (x - x.mean()) )

# Compute the covariance between x and y by hand (sigma2_xy)
sigma2_xy = (1 / len(x)) * sum( (x - x.mean()) * (y - y.mean()) )

# Compute the covariance between x and y by hand (sigma2_yx)
sigma2_yx = (1 / len(y)) * sum( (y - y.mean()) * (x - x.mean()) )

# Compute the covariance between x and y by hand (sigma2_yy)
sigma2_yy = (1 / len(y)) * sum( (y - y.mean()) * (y - y.mean()) )

# Arrange the sigma2_xx, sigma2_xy, sigma2_yx and sigma2_yy as a 2x2 matrix
np.array([ [sigma2_xx, sigma2_xy], [sigma2_yx, sigma2_yy] ])

This is the *covariance matrix* of the variables $x$ and $y$. Try to imagine how it would be, for example, the covariance matrix between three variables $x$, $y$ and $z$ (an so on for and arbitrary number of variables).

<div class="alert alert-block alert-success">
    
<b>Practice 1:</b> Get the **covariance matrix** between the variables **height** and **weight** in the "IQ" dataset.

1) In the 1<sup>st</sup> code cell below, use `np.cov()` to compute the covariance matrix between the variables `Height [m]` and `Weight [kg]` in `df`. **What happened in this first try?**
2) In the 2<sup>nd</sup> code cell below, use `np.cov()` to compute the covariance matrix between the variables `Height [m]` and `Weight [kg]` in `df_DropNan`. **What happened in this second try?**

<b>Note:</b> Uncomment and fill only those code lines with underscores `___`.

</div>

In [ ]:
# (1st code cell)

# Compute the covariance between Height [m] and Weight [kg] with NumPy and show it
#print(___(___, ___, bias=True))

# Show df's head
#df.head()

In [ ]:
# (1st code cell)

# Compute the covariance between Height [m] and Weight [kg] with NumPy and show it
print(np.cov(df['Height [m]'], df['Weight [kg]'], bias=True))

# Show df's head
df.head()

In [ ]:
# (2nd code cell)

# Drop all rows with NANs to enable covariance computation
df_DropNan = df.dropna().copy()

# Compute the covariance between Height [m] and Weight [kg] with NumPy and show it
#print(___(___, ___, bias=True))

# Show df_DropNan's head
#df_DropNan.head()

In [ ]:
# (2nd code cell)

# Drop all rows with NANs to enable covariance computation
df_DropNan = df.dropna().copy()

# Compute the covariance between Height [m] and Weight [kg] with NumPy and show it
print(np.cov(df_DropNan['Height [m]'], df_DropNan['Weight [kg]'], bias=True))

# Show df_DropNan's head
df_DropNan.head()

<div class="alert alert-block alert-success"><b>Practice 1 ends here.</b>

</div>

## The essence behind of the covariance matrix

### Standardizing the covariance matrix

Now, we will use the particular example of the covariance between `Height [m]` and `Weight [kg]` to get an intuitive understanding of this covariance matrix. As usual, let's start by visualizing the data:

In [ ]:
# Create the scatter plot with Plotnine (metric data)
gg = (
    p9.ggplot(data=df_DropNan, mapping=p9.aes(x='Height [m]', y='Weight [kg]'))
    + p9.geom_point()
    + p9.theme(figure_size=(4, 4))
)

# Show plot
gg

In order to simplify a bit some of the steps downstream, let's standardize this data:

In [ ]:
# Get the z-score for 'Height [m]' and 'Weight [kg]'
df_DropNan['Z-Height'] = (df_DropNan['Height [m]'] - np.mean(df_DropNan['Height [m]'])) / np.std(df_DropNan['Height [m]'])
df_DropNan['Z-Weight'] = (df_DropNan['Weight [kg]'] - np.mean(df_DropNan['Weight [kg]'])) / np.std(df_DropNan['Weight [kg]'])

# Show df_DropNan's head
df_DropNan.head()

In [ ]:
# Create the scatter plot with Plotnine (z-score data)
gg = (
    p9.ggplot(data=df_DropNan, mapping=p9.aes(x='Z-Height', y='Z-Weight'))
    + p9.geom_point()
    + p9.theme(figure_size=(4, 4))
    + p9.geom_vline(xintercept=0, color='darkgrey')
    + p9.geom_hline(yintercept=0, color='darkgrey')
)

# Show plot
gg

Note that both scatter plots are identical, with the unique difference being the scale of both axes. Now, let's compute the covariance matrix for the z-score data we just generated.

In [ ]:
# Compute the covariance between 'Z-Height' and 'Z-Weight' with NumPy and show it
Sigma_Z = np.cov(df_DropNan['Z-Height'], df_DropNan['Z-Weight'], bias=True)

# Show the covariance matrix
Sigma_Z

Note how the *covariance matrix* of the z-score data has *ones* in the diagonal in contrast to its non z-scored counterpart:

In [ ]:
# Compute the covariance between 'Height' and 'Weight' with NumPy and show it
Sigma = np.cov(df_DropNan['Height [m]'], df_DropNan['Weight [kg]'], bias=True)

# Show the covariance matrix
Sigma

This is something natural because, when we z-scored the data, we manually bring the distribution of values to a standardized distribution with variance 1 and mean 0.

### Eigenthings of the covariance matrix

Now, if we look at the covariance matrix through the prism of *Linear Algebra*, we could understand this $2 \times 2$ matrix as a *linear transformation in a two-dimensional space*... Remember that *linear transformation* ($\textbf{M}$) is just something that "transmutes" an input vector ($\vec{v}$) to an output vector ($\vec{u}$). Let's see an example:

In [ ]:
# Initiate a "seeded" random number generator
rnd_seed = np.random.default_rng(seed=1985)

# Create a random vector "v" (2 x 1)
v = rnd_seed.integers(low=-5, high=5, size=2)

# Create a random lineal transformation "M" (2 x 2)
M = rnd_seed.integers(low=-5, high=5, size=(2, 2))

# Apply "M" to "v" and store the output vector "u" (2 x 1)
u = M @ v

# Print "v", "M" and "u"
print(f'v = {v}', f'M = {M}', f'u = {u}', sep='\n\n')

\begin{equation}
\textbf{M} \vec{v} = \begin{pmatrix}  -4 & -1 \\ -1 & 0 \end{pmatrix} \cdot \begin{pmatrix} -2 \\ -3 \end{pmatrix} = \begin{pmatrix} (-4) \cdot (-2) + (-1) \cdot (-3) \\ (-1) \cdot (-2) + 0 \cdot (-3) \end{pmatrix} = \begin{pmatrix} 8 + 3 \\ 2 + 0 \end{pmatrix} = \begin{pmatrix} 11 \\ 2  \end{pmatrix} = \vec{u}
\end{equation}

These "transmutations" cannot be arbitrary as they have to preserve certain vector operations (vector additivity and vector scalar multiplication). Intuitively, after passing through a linear transformation, the original direction of $\vec{v}$ can be (but no necessarily) *rotated* and the original length of $\vec{v}$ can be (but no necessarily) *elongated* or *shortened*:

In [ ]:
# Plot input "v" (red) and output "u" (green) vectors
gg = (
    p9.ggplot(data=df)
    + p9.geom_segment(
        x=0, y=0,
        xend=v[0], yend=v[1],
        color='red',
        arrow=p9.arrow(angle=30, length=0.05, ends="last", type="closed")
     )
    + p9.geom_segment(
        x=0, y=0,
        xend=u[0], yend=u[1],
        color='green',
        arrow=p9.arrow(angle=30, length=0.05, ends="last", type="closed")
     )
    + p9.geom_vline(xintercept=0, color='darkgrey')
    + p9.geom_hline(yintercept=0, color='darkgrey')
    + p9.xlim(-3, 11) + p9.ylim(-3, 11) + p9.xlab('x') + p9.ylab('y')
    + p9.theme(figure_size=(4, 4))
)

# Show plot
gg

Linear transformations present certain very special directions (two in the particular case of two-dimensional linear transformations). All the vectors living in these special directions will keep their directions unchanged after passing through the linear transformation. In other words, these vectors can be *elongated* or *shortened* but never *rotated* when passing through the linear transformation. This is precisely the definition of *eigenvector*: a vector $\vec{v}$ that is proportional to itself, $\lambda \vec{v}$, after passing through the linear transformation $\textbf{M}$:

\begin{equation}
\textbf{M} \vec{v} = \vec{u} = \lambda \vec{v},
\end{equation}

where this proportional constant $\lambda$ is the *eigenvalue* associated to the eigenvector $\vec{v}$. Let's get the eigenvectors and eigenvalues of $\textbf{M}$ leveraging the **NumPy** function [`np.linalg.eig()`](https://numpy.org/doc/2.3/reference/generated/numpy.linalg.eig.html):

In [ ]:
# Get the eigenvectors and eigenvalues of M
eigenthings = np.linalg.eig(M)

# Print the eigenvectors and eigenvalues of M
print(eigenthings)

<div class="alert alert-block alert-danger">

<b>Caveat:</b> Keep in mind that `np.linalg.eig()` return the eigenvectors in a matrix <b><u>as columns</u></b>, (not as rows!):

\begin{equation}
 \begin{pmatrix} \vec{v}_1 & \vec{v}_2 \end{pmatrix} =
 \begin{pmatrix} \begin{pmatrix} v_{11} \\ v_{21} \end{pmatrix} & \begin{pmatrix} v_{12} \\ v_{22} \end{pmatrix} \end{pmatrix} =
  \begin{pmatrix} \begin{pmatrix} -0.9732 \\ -0.2298 \end{pmatrix} & \begin{pmatrix} 0.2298 \\ -0.9732 \end{pmatrix} \end{pmatrix} =
 \begin{pmatrix} -0.9732 & 0.2298 \\ -0.2298 & -0.9732 \end{pmatrix}
 \Rightarrow  \begin{cases}
      \vec{v}_1 = \begin{pmatrix} -0.9732 \\ -0.2298 \end{pmatrix} \\
      \\
      \vec{v}_2 = \begin{pmatrix} 0.2298 \\ -0.9732 \end{pmatrix} \\
    \end{cases} 
\end{equation}

</div>

In [ ]:
# Store the first (0) and the second (1) eigenvectors
e0 = eigenthings.eigenvectors[:, 0]
e1 = eigenthings.eigenvectors[:, 1]
print('Eigenvectors:', e0, e1, '', sep='\n')

# Store the first (0) and the second (1) eigenvalues
λ0 = eigenthings.eigenvalues[0]
λ1 = eigenthings.eigenvalues[1]
print('Eigenvalues:', λ0, λ1, '', sep='\n')

# Apply "M" to the first (0) and the second (1) eigenvectors and store them
Me0 = M @ e0
Me1 = M @ e1
print('M applied to the eigenvectors:', Me0, Me1, '', sep='\n')

# Multiply each eigenvector by its eigenvalue
λ0e0 = λ0 * e0
λ1e1 = λ1 * e1
print('Eigenvalues multiplied by the eigenvectors:', λ0e0, λ1e1, '', sep='\n')

Note how the product between the eigenvectors and their corresponding eigenvalues, $\lambda_{i} \vec{e}_i$, returns the vectors resulting when we applying the linear transformation to such eigenvector, $\textbf{M} \vec{e}_i$, because, by construction:

\begin{equation}
\textbf{M} \vec{e}_i = \lambda_{i} \vec{e}_i
\end{equation}

In [ ]:
# Plot input "e0" (red) and output "Me0" (green) vectors
gg = (
    p9.ggplot(data=df)
    + p9.geom_segment(
        x=0, y=0,
        xend=e0[0], yend=e0[1],
        color='red',
        arrow=p9.arrow(angle=30, length=0.05, ends="last", type="closed")
     )
    + p9.geom_segment(
        x=0, y=0,
        xend=Me0[0], yend=Me0[1],
        color='green',
        arrow=p9.arrow(angle=30, length=0.05, ends="last", type="closed")
     )
    + p9.geom_vline(xintercept=0, color='darkgrey')
    + p9.geom_hline(yintercept=0, color='darkgrey')
    + p9.xlim(-1, 5) + p9.ylim(-1, 5) + p9.xlab('x') + p9.ylab('y')
    + p9.theme(figure_size=(4, 4))
    
)

# Show plot
gg

In [ ]:
# Plot input "e1" (red) and output "Me1" (green) vectors
gg = (
    
    # Create the scatter plot with Plotnine
    p9.ggplot(data=df)
    + p9.geom_segment(
        x=0, y=0,
        xend=e1[0], yend=e1[1],
        color='red',
        arrow=p9.arrow(angle=30, length=0.05, ends="last", type="closed")
     )
    + p9.geom_segment(
        x=0, y=0,
        xend=Me1[0], yend=Me1[1],
        color='green',
        arrow=p9.arrow(angle=30, length=0.05, ends="last", type="closed")
     )
    + p9.geom_vline(xintercept=0, color='darkgrey')
    + p9.geom_hline(yintercept=0, color='darkgrey')
    + p9.xlim(-1, 1) + p9.ylim(-1, 1) + p9.xlab('x') + p9.ylab('y')
    + p9.theme(figure_size=(4, 4))
)

# Show plot
gg

We see that, after passing through $\textbf{M}$, the first eigenvector $\vec{e}_0$ is *reversed* and *elongated* ($\lambda_0 = -4.236$) whilst the second eigenvector $\vec{e}_1$ is *shortened* ($\lambda_1 = +0.236$), but both of them keep their directions intact. Note how their corresponding eigenvalues encode this outcome.

<div class="alert alert-block alert-success">
    
<b>Practice 2:</b> Get the **eigenvalues** and the **eigenvectors** of the **covariance matrix** that we have already obtained for the **height** and **weight** after z-scoring the data.

1) In the 1<sup>st</sup> code cell below, use `np.linalg.eig()` to compute the the eigenvalues and the eigenvectors of the covariance matrix between the variables `Z-Height` and `Z-Weight` from our `df_DropNan` dataset (`Sigma_Z`).
2) In the 2<sup>nd</sup> code cell below, breakdown the `EigResult` object returned by `np.linalg.eig()` and store both eigenvectors as `e0` and `e1` and both eigenvalues as `λ0` and `λ1`.
3) In the 3<sup>rd</sup> code cell below, represent `e0` and `e1` on the scatter plot with `Z-Weight` versus `Z-Height`. **What do you observe?**
4) In the 4<sup>th</sup> code cell below, represent again `e0` and `e1` on the scatter plot, but scaling each eigenvector by its corresponding eigenvalue. **What do you observe now?**

<b>Note:</b> Uncomment and fill only those code lines with underscores `___`.

</div>

In [ ]:
# (1st code cell)

# Show the the z-scored covariance matrix
# print('Covariance matrix:', ___, '', sep='\n')

# Get the eigenvectors and eigenvalues of the z-scored covariance matrix
# eigen = ___(___)

# Print the eigenvectors and eigenvalues of the z-scored covariance matrix
# print('EigResult object:', eigen, '', sep='\n')

In [ ]:
# TODO

In [ ]:
# (1st code cell)

# Show the the z-scored covariance matrix
print('Covariance matrix:', Sigma_Z, '', sep='\n')

# Get the eigenvectors and eigenvalues of the z-scored covariance matrix
eigen = np.linalg.eig(Sigma_Z)

# Print the eigenvectors and eigenvalues of the z-scored covariance matrix
print('EigResult object:', eigen, '', sep='\n')

In [ ]:
# (2nd code cell)

# Store the first (0) and the second (1) eigenvectors and show them
# e0 = eigen.___[___, ___]
# e1 = eigen.___[___, ___]
# print('Eigenvectors:', ___, ___, '', sep='\n')

# Store the first (0) and the second (1) eigenvalues and show them
# λ0 = eigen.___[___]
# λ1 = eigen.___[___]
# print('Eigenvalues:', ___, ___, '', sep='\n')

In [ ]:
# (2nd code cell)

# Store the first (0) and the second (1) eigenvectors and show them
e0 = eigen.eigenvectors[:, 0]
e1 = eigen.eigenvectors[:, 1]
print('Eigenvectors:', e0, e1, '', sep='\n')

# Store the first (0) and the second (1) eigenvalues and show them
λ0 = eigen.eigenvalues[0]
λ1 = eigen.eigenvalues[1]
print('Eigenvalues:', λ0, λ1, '', sep='\n')

In [ ]:
# (3rd code cell)

# Create the scatter plot with Plotnine (z-score data)
gg = (
    p9.ggplot(data=df_DropNan, mapping=p9.aes(x='Z-Height', y='Z-Weight'))
    + p9.geom_point()
    + p9.theme(figure_size=(4, 4))

     # Plot the first eigenvector
#    + p9.geom_segment(
#        x=0, y=0,
#        xend=___, yend=___],
#        color='red',
#        arrow=p9.arrow(angle=30, length=0.05, ends="last", type="closed")
#     )

     # Plot the second eigenvector
#    + p9.geom_segment(
#        x=0, y=0,
#        xend=___, yend=___,
#        color='green',
#        arrow=p9.arrow(angle=30, length=0.05, ends="last", type="closed")
#     )

    + p9.geom_vline(xintercept=0, color='darkgrey')
    + p9.geom_hline(yintercept=0, color='darkgrey')
)

# Show plot
gg

In [ ]:
# (3rd code cell)

# Create the scatter plot with Plotnine (z-score data)
gg = (
    p9.ggplot(data=df_DropNan, mapping=p9.aes(x='Z-Height', y='Z-Weight'))
    + p9.geom_point()
    + p9.theme(figure_size=(4, 4))

    # Plot the first eigenvector
    + p9.geom_segment(
        x=0, y=0,
        xend=e0[0], yend=e0[1],
        color='red',
        arrow=p9.arrow(angle=30, length=0.05, ends="last", type="closed")
     )

    # Plot the second eigenvector
    + p9.geom_segment(
        x=0, y=0,
        xend=e1[0], yend=e1[1],
        color='green',
        arrow=p9.arrow(angle=30, length=0.05, ends="last", type="closed")
     )
    + p9.geom_vline(xintercept=0, color='darkgrey')
    + p9.geom_hline(yintercept=0, color='darkgrey')
)

# Show plot
gg

In [ ]:
# (4th code cell)

# Create the scatter plot with Plotnine (z-score data)
gg = (
    p9.ggplot(data=df_DropNan, mapping=p9.aes(x='Z-Height', y='Z-Weight'))
    + p9.geom_point()
    + p9.theme(figure_size=(4, 4))

    # Plot the second eigenvector (scaled)
#    + p9.geom_segment(
#        x=0, y=0,
#        xend=___, yend=___,
#        color='red',
#        arrow=p9.arrow(angle=30, length=0.05, ends="last", type="closed")
#     )
#
#    # Plot the second eigenvector (scaled)
#    + p9.geom_segment(
#        x=0, y=0,
#        xend=___, yend=___,
#        color='green',
#        arrow=p9.arrow(angle=30, length=0.05, ends="last", type="closed")
#     )

    + p9.geom_vline(xintercept=0, color='darkgrey')
    + p9.geom_hline(yintercept=0, color='darkgrey')
)

# Show plot
gg

In [ ]:
# (4th code cell)

# Create the scatter plot with Plotnine (z-score data)
gg = (
    p9.ggplot(data=df_DropNan, mapping=p9.aes(x='Z-Height', y='Z-Weight'))
    + p9.geom_point()
    + p9.theme(figure_size=(4, 4))

    + p9.geom_segment(
        x=0, y=0,
        xend=λ0*e0[0], yend=λ0*e0[1],
        color='red',
        arrow=p9.arrow(angle=30, length=0.05, ends="last", type="closed")
     )
    + p9.geom_segment(
        x=0, y=0,
        xend=λ1*e1[0], yend=λ1*e1[1],
        color='green',
        arrow=p9.arrow(angle=30, length=0.05, ends="last", type="closed")
     )
    + p9.geom_vline(xintercept=0, color='darkgrey')
    + p9.geom_hline(yintercept=0, color='darkgrey')
)

# Show plot
gg

<div class="alert alert-block alert-success"><b>
Practice 2 ends here.</b>
</div>

### The meaning of the covariance matrix

So far, given two z-scored columns of our "IQ" dataset (`'Z-Height'` and `'Z-Weight'`), we have been able to compute the covariance matrix and to find its corresponding eigenvectors and eigenvalues . At the end of **Practice 2**, we intuitively verified a crucial insight about the covariance matrix: the **eigenvector with the largest eigenvalue** (in absolute value) is denoting the **direction along which our data exhibits the largest variability** contribution, and so on, the eigenvector with the second largest eigenvalue (in absolute value) is denoting the direction along which our data exhibits the second largest variability contribution. This is the key principle behind the [**Principal Component Analysis**](https://en.wikipedia.org/wiki/Principal_component_analysis) (PCA).

Having said that, now, let's organize the eigenvalues and eigenvectors of the z-scored covariance matrix `Sigma_Z` as a nice list of `(eigenvalues, eigenvector)` tuples sorted in decreasing order according the absolute values of their corresponding eigenvalues:

In [ ]:
# Make a nice list of (|eigenvalue|, eigenvector) "eigentuples"
eigentuples = [(np.abs(eigen.eigenvalues[i]), eigen.eigenvectors[:, i]) for i in range(len(eigen.eigenvalues))]

# Show the list of "eigentuples"
eigentuples

In [ ]:
# Sort the "eigentuples" from the highest to the lowest eigenvalue absolute value
eigentuples = sorted(eigentuples, reverse=True)

# Show the list of sorted "eigentuples"
eigentuples

<div class="alert alert-block alert-info"><b>Tip:</b>
In our particular case, the tuples are already sorted, but we decided to "sort" them anyway so we don't loose any generality in our workflow.
</div>

## Principal component analysis

We almost have all the pieces needed to perform our first PCA! The last step is the **transformation of our data** via the **projection matrix**, which is simply the matrix laying out the sorted eigenvectors:

\begin{equation}
 \Pi = \begin{pmatrix} \vec{e}_1 & \vec{e}_2 \end{pmatrix} =
 \begin{pmatrix} \begin{pmatrix} e_{11} \\ e_{21} \end{pmatrix} & \begin{pmatrix} e_{12} \\ e_{22} \end{pmatrix} \end{pmatrix} =
 \begin{pmatrix} \begin{pmatrix} 0.7071 \\ 0.7071 \end{pmatrix} & \begin{pmatrix} -0.7071 \\ 0.7071 \end{pmatrix} \end{pmatrix} =
 \begin{pmatrix} 0.7071 & -0.7071 \\ 0.7071 & 0.7071 \end{pmatrix}
\end{equation}

Let's get this projection matrix:

In [ ]:
# Prepare the projection matrix with all eigenvectors
Π = np.transpose(np.array([eigenvector for (eigenvalue, eigenvector) in eigentuples]))

# Show the shape of the projection matrix
print('Projection matrix shape:', Π.shape, '', sep='\n')

# Show the projection matrix
print('Projection matrix:', Π, '', sep='\n')

<div class="alert alert-block alert-danger">

<b>Caveat:</b> Remember, eigenvector <b><u>as columns</u></b> of the projection matrix, (not as rows!):

</div>

In [ ]:
# Transform our data: (38 x 2) @ (2 x 2) = (38 x 2)
df_DropNan_T = df_DropNan[['Z-Height', 'Z-Weight']] @ Π

# Show the shape of the transformed data
print('Projection matrix shape:', df_DropNan_T.shape, '', sep='\n')

# Show the transformed data
df_DropNan_T.head()

Finally, we can use the dataframe with the transformed data `df_DropNan_T` to represent our original z-scored data in the coordinate basis of the eigenvectors of the covariance matrix sorted in decreasing order according to their corresponding eigenvalues (in absolute value) or, in other words, **represent our original z-scored data via their principal components** (two principal components in this particular case). Let's change the column names in `df_DropNan_T`:

In [ ]:
# Switch col names from correlative integers, to correlative PC labels
df_DropNan_T.columns = [f'PC{str(colname+1)}' for colname in list(df_DropNan_T)]

# Show the transformed data
df_DropNan_T.head()

Now let's get our first PCA plot!

In [ ]:
# Plot the PCA
gg = (
    p9.ggplot(data=df_DropNan_T, mapping=p9.aes(x='PC1', y='PC2'))
    + p9.geom_point()
    + p9.theme(figure_size=(4, 4))

    + p9.geom_vline(xintercept=0, color='green')
    + p9.geom_hline(yintercept=0, color='red')
)

# Show plot
gg

Intuitively, we observe that the space where our original data points used to live have been rotated (clockwise) and stretched (vertically) after the transformation. Now the directions of the <span style="color:red">**1st eigenvector (PC1)**</span> and the <span style="color:green">**2nd eigenvector (PC2)**</span> appear totally vertical and horizontal, respectively:

In [ ]:
# Create the scatter plot with Plotnine (z-score data)
gg = (
    p9.ggplot(data=df_DropNan, mapping=p9.aes(x='Z-Height', y='Z-Weight'))
    + p9.geom_point()
    + p9.theme(figure_size=(4, 4))

    + p9.geom_segment(
        x=0, y=0,
        xend=λ0*e0[0], yend=λ0*e0[1],
        color='red',
        arrow=p9.arrow(angle=30, length=0.05, ends="last", type="closed")
     )
    + p9.geom_segment(
        x=0, y=0,
        xend=λ1*e1[0], yend=λ1*e1[1],
        color='green',
        arrow=p9.arrow(angle=30, length=0.05, ends="last", type="closed")
     )
    + p9.geom_vline(xintercept=0, color='darkgrey')
    + p9.geom_hline(yintercept=0, color='darkgrey')
)

# Show plot
gg

Remember, the <span style="color:red">**PC1**</span> is the component (direction) along which our data exhibits the top 1 variability contribution, the <span style="color:green">**PC2**</span> the top 2 variability contribution and so on (PC3, PC4, ..., PCN) in case of having more complex datasets. Let's quickly compute each one of these variability contributions and nicely label them in our PCA:

In [ ]:
# Take the eigenvalues we already sorted
eig_vals_sorted = [eigenvalue for (eigenvalue, eigenvector) in eigentuples]
print(eig_vals_sorted)

# Compute the variance proportion explained by each eigenvector (AKA principal component)
list_expvar = (eig_vals_sorted / sum(eig_vals_sorted)) * 100
print(list_expvar)

# Round estimates for better labeling below
list_expvar_round = [round(expvar, 2) for expvar in list_expvar]

# Plot the PCA
gg = (
    p9.ggplot(data=df_DropNan_T, mapping=p9.aes(x='PC1', y='PC2'))
    + p9.geom_point()
    + p9.theme(figure_size=(4, 4))

    + p9.geom_vline(xintercept=0, color='green')
    + p9.geom_hline(yintercept=0, color='red')

    + p9.xlab(f'PC1 ({list_expvar_round[0]}%)') + p9.ylab(f'PC2 ({list_expvar_round[1]}%)')
)

# Show plot
gg

The <span style="color:red">**PC1**</span> accounts for 85% of the dataset variability and the <span style="color:green">**PC2**</span> accounts for 15% of the dataset variability, which in total represent the totality of the dataset variability (100%).

## Dimensionality reduction

The true power of a PCA is on its [**dimensionality reduction**](https://en.wikipedia.org/wiki/Dimensionality_reduction) capabilities. At the moment, we worked with an simple example with just 2 magnitudes, AKA **features**, (`Z-Height` and `Z-Weight`) that were recorder for 38 individuals, AKA **samples**, comprising a dataset $D$ of dimension $(38 \times 2)$. After transforming $D$ via the projection matrix $\Pi$ of dimension $(2 \times 2)$, we got a transformed dataset $T$ of dimension $(38 \times 2) \cdot (2 \times 2) = (38 \times 2)$, which is exactly the same dimension of $D$. Therefore, in practice we did not obtain a dimensionality reduction when doing $D \cdot \Pi = T$.

<div class="alert alert-block alert-info"><b>Tip:</b>
Typically, the number of <b>features</b> is orders of magnitude higher than the number of <b>samples</b>. For example, a typical proteomics dataset has $\sim 10^{3}$ quantified proteins (features) across $\sim 10^{1}$ patients (samples), which represent a dataset dimensionality of $(\sim 10^{1} \times \sim 10^{3})$.
</div>

## PCA step by step (from scratch)

Let's unleash the true power of the PCA by working with *all the features* present in our "IQ" Dataset. We will repeat all the major steps already shown for the `Height [m]` and `Weight [kg]` sub-dataset:
1) Standardize data.
2) Get covariance matrix and its eigenthings.
3) Sort PCs and compute explained variance per PC.
4) Get the projection matrix and define number of PCs to keep.
5) Transform the data.
6) Visualize the PCA.

### 1 Standardize data

As usual, we will drop those samples with missing values in any feature. In addition, we will also exclude the qualitative feature `'Gender'` from our PCA (later on we will talk about this qualitative feature).

In [ ]:
# Drop all samples with NANs to enable covariance matrix computation
D = df.dropna()

# Drop all qualitative features
D = D.drop(columns=['Gender'])

# Show D's head
D.head()

In [ ]:
# Create User-Defined Function to compute z-scores
def zscore(x):
    return (x - np.mean(x)) / np.std(x)

# Standardize dataset across samples (rows) (axis='index')
DZ = D.apply(zscore, axis='index')

# Show DZ's head
DZ.head()

### 2 Get covariance matrix and its eigenthings

Now, we will get the covariance matrix between all features in our dataset. Since we have 6 features, the resulting covariance matrix must have dimension $(6 \times 6)$.

In [ ]:
# Compute the covariance matrix between all our features
Σ = np.cov(DZ.T, bias=True)

# Get eigenvectors and eigenvalues of the covariance matrix
eigen = np.linalg.eig(Σ)

# Make a nice list of (|eigenvalue|, eigenvector) "eigentuples"
eigentuples = [(np.abs(eigen.eigenvalues[i]), eigen.eigenvectors[:, i]) for i in range(len(eigen.eigenvalues))]

# Show our (|eigenvalue|, eigenvector) "eigentuples"
eigentuples

### 3 Sort PCs and compute explained variance per PC

We now have 6 "eigentuples", one for each feature in our dataset. Let's sort them in decreasing order according the absolute value of their corresponding eigenvalues to establish the 6 PCs:

In [ ]:
# Sort the "eigentuples" from the highest to the lowest eigenvalue absolute value
eigentuples_sorted = sorted(eigentuples, reverse=True)

# Show the list of sorted "eigentuples"
eigentuples_sorted

And let's compute the variance proportion explained by each PC:

In [ ]:
# Take the sorted eigenvalues from the sorted "eigentuples"
eig_vals_sorted = [eigenvalue for (eigenvalue, eigenvector) in eigentuples_sorted]

# Compute the variance proportion explained by each PC
list_expvar = (eig_vals_sorted / sum(eig_vals_sorted)) * 100

# Round variance proportion estimates for better labeling below
list_expvar_round = [round(expvar, 2) for expvar in list_expvar]
print(list_expvar_round)

In [ ]:
# Prepare an "Explined Variance" dataframe to visualize this data
df_expvar = pd.DataFrame({
    'PC': np.arange(1, len(list_expvar) + 1, 1),
    'Explained variance (%)': list_expvar,
    'Accumulated explained variance (%)': np.cumsum(list_expvar)
})

# Show df_expvar's
df_expvar

In [ ]:
# Plot the accumulated explained variance plot
gg = (
    p9.ggplot(data=df_expvar, mapping=p9.aes(x='PC', y='Accumulated explained variance (%)'))
    + p9.geom_point() + p9.geom_line()
    + p9. scale_x_continuous(breaks=np.arange(1, len(list_expvar) + 1, 1))
    + p9.geom_vline(xintercept=4.5, color='darkgrey', linetype='dashed')
)

# Show plot
gg

Note that the first two PCs (PC1 and PC2) are enough to explain more than the 85% of overall dataset variance. In general, we can always define a cut and drop those PCs that don't capture a relevant contribution to the overall dataset variance. In this particular example, we observe that PC1 to PC4 explain 99% of overall dataset variance and therefore, we could effectively ignore PC5 to PC6 from now onward.

### 4 Get the projection matrix and define number of PCs to keep

We are almost there! First let's make the full projection matrix $\Pi$, to then define an arbitrary cut to keep a limited number of PCs that explain the big majority of the overall dataset variance:

In [ ]:
# Prepare the projection matrix with all eigenvectors
Π = np.transpose(np.array([eigenvector for (eigenvalue, eigenvector) in eigentuples]))

# Show the projection matrix
print('Projection matrix:', Π, '', sep='\n')

# Show the shape of the projection matrix
print('Projection matrix shape:', Π.shape, '', sep='\n')

# Define number of top-k PCs to keep
k = 2

# Select top-k eigenvectors of the projection matrix
Π_cut = Π[:, :k]

# Show the projection matrix (after cutting)
print('Projection matrix (after cutting):', Π_cut, '', sep='\n')

# Show the shape of the projection matrix (after cutting)
print('Projection matrix shape (after cutting):', Π_cut.shape, '', sep='\n')

### 5 Transform the data

To finally transform our data via the "cut" projection matrix (and therefore reduce the dimensionality of our dataset) we must multiply their corresponding matrices:

\begin{equation}
D_{z-scored} \Pi_{cut} = D_{\rm{transformed}}
\end{equation}

<div class="alert alert-block alert-info">
<b>Tip:</b> Remember that he product between two matrices can only be defined if the number of columns of the first matrix matches the number of rows of the second matrix: $(m \times n) \cdot (n \times p)$. If the dimensions of the multiplying matrices are $(m \times n)$ and $(n \times p)$, the product matrix will have dimension $(m \times p)$.
</div>

Can we really get the product between $D_{z-scored}$ and $\Pi_{cut}$? Let's check if their shapes match:

In [ ]:
# Show the shape of the data matrix (z-scored)
print('Data (z-scored) matrix shape:', DZ.shape, '', sep='\n')

# Show the shape of the projection matrix (cut)
print('Data matrix shape:', Π_cut.shape, '', sep='\n')

Everything looks correct so let's do it:

In [ ]:
# Transform our data: (38 x 6) @ (6 x 2) = (38 x 2)
DZT = DZ @ Π_cut

# Show the shape of the transformed data
print('Projection matrix shape:', DZT.shape, '', sep='\n')

# Show the transformed data
DZT.head()

Note how we reduced the dimensionality from $(38 \times 6)$ to $(38 \times 2)$.

In [ ]:
# Switch col names from correlative integers, to correlative PC labels
DZT.columns = [f'PC{str(colname+1)}' for colname in list(DZT)]

# Show the transformed data
DZT.head()

### 6 Visualize the PCA

We can finally visualize any pairwse-PC scatter plot. Usually PC2 vs PC1 is the most informative comparison but, sometimes, certain (more subtle) variability sources can be captured by "superior" PCs.

In [ ]:
# Plot the PCA
gg = (
    p9.ggplot(data=DZT, mapping=p9.aes(x='PC1', y='PC2'))
    + p9.geom_point()
    + p9.theme(figure_size=(4, 4))
    + p9.xlab(f'PC1 ({list_expvar_round[0]}%)') + p9.ylab(f'PC2 ({list_expvar_round[1]}%)')
)

# Show plot
gg

In order to prepare the incoming **Practice 3**, we added the `PC1` and `PC2` data we just obtained to the original dataset stored in `df_DropNan` dataframe:

In [ ]:
# Merge original data with the PC1 and PC2 data we just obtained
df_pca = df_DropNan.merge(DZT, left_index=True, right_index=True)
df_pca.drop(columns=['Z-Height', 'Z-Weight'], inplace=True)
df_pca.head()

<div class="alert alert-block alert-success">
    
<b>Practice 3:</b> Get a deeper insight of the PCA obtained for the "IQ" dataset:

1) Look at the PCA in the 1<sup>st</sup> code cell below. **What do you think is the main source of variability captured by PC2?**
2) Look at the PCA in the 2<sup>nd</sup> code cell below. **What do you think is the main source of variability captured by PC1?**
3) **How do you interpret the two clusters of samples appearing in the PCA?**

<b>Note:</b> Uncomment and fill only those code lines with underscores `___`.

</div>

In [ ]:
# (1st code cell)

# Define variable to represent in the PCA as shape and color
pca_shape = 'Gender'
pca_color = 'Weight [kg]'

# Plot the PCA
gg = (
    p9.ggplot(data=df_pca, mapping=p9.aes(x='PC1', y='PC2', shape=pca_shape, color=pca_color))
    + p9.geom_point()
    + p9.theme(figure_size=(4.5, 4))
    + p9.xlab(f'PC1 ({list_expvar_round[0]}%)') + p9.ylab(f'PC2 ({list_expvar_round[1]}%)')
)

# Show plot
gg

In [ ]:
# (2nd code cell)

# Define variable to represent in the PCA as shape and color
pca_shape = 'Gender'
pca_color = 'FSIQ'

# Plot the PCA
gg = (
    p9.ggplot(data=df_pca, mapping=p9.aes(x='PC1', y='PC2', shape=pca_shape, color=pca_color))
    + p9.geom_point()
    + p9.theme(figure_size=(4.5, 4))
    + p9.xlab(f'PC1 ({list_expvar_round[0]}%)') + p9.ylab(f'PC2 ({list_expvar_round[1]}%)')
)

# Show plot
gg

<div class="alert alert-block alert-success"><b>
Practice 3 ends here.</b>
</div>

## PCA (with scikit-learn)

There is a **Python** package called [**scikit-learn**](https://scikit-learn.org/stable/) with all the tools you need to get a PCA in just a few code lines. Look how we can implement everything we did in the previous section in just a few code lines:

In [ ]:
# Load the specific classees from the scikit-learn package with its corresponding alias
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

# Drop all samples with NANs to enable covariance matrix computation
D_skl = df.dropna()

# Drop all qualitative features
D_skl = D_skl.drop(columns=['Gender'])

# 1) Standardize data.
DZ_skl = StandardScaler().fit_transform(D_skl)

# 2) Get covariance matrix and its eigenthings.
# 3) Sort PCs and compute explained variance per PC. 
# 4) Get the projection matrix and define number of PCs to keep.
pca = PCA(n_components=2)
pca.fit(DZ_skl)
list_expvar_round_skl = [round(expvar * 100, 2) for expvar in pca.explained_variance_ratio_]

# 5) Transform the data.
DZT_skl_array = pca.transform(DZ_skl)

# Switch col names from correlative integers, to correlative PC labels
DZT_skl = pd.DataFrame(DZT_skl_array)
DZT_skl.columns = [f'PC{str(colname+1)}' for colname in list(DZT_skl)]

# 6) Visualize the PCA.
gg = (
    p9.ggplot(data=DZT_skl, mapping=p9.aes(x='PC1', y='PC2'))
    + p9.geom_point()
    + p9.theme(figure_size=(4, 4))
    + p9.xlab(f'PC1 ({list_expvar_round_skl[0]}%)') + p9.ylab(f'PC2 ({list_expvar_round_skl[1]}%)')
)

# Show plot
gg

From now onward, to save time and stick to the standard way to work, always use this approach to perform your PCAs. In any case, don't forget the true essence behind a "basic" PCA (covariance matrix, eigenthings, projection matrix, etc...).